# Libraries declaration:

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from ast import literal_eval
import pandas as pd
from random import randint
from time import sleep
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

***Getting all product categories:***

In [ ]:
main_URL = "https://www.pricez.co.il/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(main_URL, headers=headers)
print(page.status_code)
soup = BeautifulSoup(page.content, "html.parser")
category_URL_arr1 = soup.find_all('div', attrs={'onclick': re.compile('window.location*')})
category_URL_arr2 = [link.attrs['onclick'] for link in category_URL_arr1]
del category_URL_arr2[-3:-1]  # those 2 categories are not needed (not a category)
category_URL_arr3 = {link[link.rindex('/')+1:-1] : main_URL + link[link.index("/Category/"): -1] for link in category_URL_arr2}


***Getting all tabs for each categories:***

In [ ]:
for Key, URL in category_URL_arr3.items():
    page = requests.get(URL , headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    tab_URL_arr_temp1 = soup.find_all('a', attrs={'href': re.compile('#Tab*')})
    tab_URL_arr_temp2 = [URL + link.attrs['href'] for link in tab_URL_arr_temp1]
    category_URL_arr3[Key] = {tab_URL_arr_temp1[index].text : tab_URL_arr_temp2[index] for index in range(len(tab_URL_arr_temp2))}


***Getting all sub tabs under each tab for each categories:***

In [ ]:
for Key_Layer1, URL_Layer1 in category_URL_arr3.items():
    for Key_Layer2, URL_Layer2 in category_URL_arr3[Key_Layer1].items():
        URL = URL_Layer2
        page = requests.get(URL, headers=headers)
        soup = BeautifulSoup(page.content, "html.parser")
        sub_tab_URL_arr_temp1 = soup.find_all('div', attrs={'id': re.compile(URL[URL.rindex('#')+1:])})
        sub_tab_URL_arr_temp1 = sub_tab_URL_arr_temp1[0].find_all('a', attrs={'href': re.compile('/Category/*')})
        category_URL_arr3[Key_Layer1][Key_Layer2] = {link.text: main_URL + link.attrs['href'] for link in sub_tab_URL_arr_temp1}


***Getting all products sub tab for each categories:***

In [ ]:
for Key_Layer1, URL_Layer1 in category_URL_arr3.items():
    for Key_Layer2, URL_Layer2 in category_URL_arr3[Key_Layer1].items():
        for Key_Layer3, URL_Layer3 in category_URL_arr3[Key_Layer1][Key_Layer2].items():
            page = requests.get(URL_Layer3, headers=headers)
            sleep(randint(2, 5))
            soup = BeautifulSoup(page.content, "html.parser")
            prod_URL_arr_temp1 = soup.find_all('div', attrs={'class': re.compile('ProductName*')})
            prod_URL_arr_temp2 = list()
            for temp1 in prod_URL_arr_temp1:
                prod_URL_arr_temp2.append(temp1.find_all('a', attrs={'href': re.compile('/Product/*')}))
            category_URL_arr3[Key_Layer1][Key_Layer2][Key_Layer3] = {link[0].text: main_URL + link[0].attrs['href'] for link in prod_URL_arr_temp2}


***Getting all information of all the products:***

In [ ]:
for Key_Layer1, URL_Layer1 in category_URL_arr3.items():
    for Key_Layer2, URL_Layer2 in category_URL_arr3[Key_Layer1].items():
        for Key_Layer3, URL_Layer3 in category_URL_arr3[Key_Layer1][Key_Layer2].items():
            sleep(randint(45,60))
            for Key_Layer4, URL_Layer4 in category_URL_arr3[Key_Layer1][Key_Layer2][Key_Layer3].items():
                page = requests.get(URL_Layer4, headers=headers)
                sleep(randint(3, 6))
                soup = BeautifulSoup(page.content, "html.parser")
                prod_arr1 = soup.find_all('iframe')
                prod_arr1 = [attr.attrs['src'] for attr in prod_arr1]
                URL = main_URL[:-1] + prod_arr1[0]
                page = requests.get(URL)
                sleep(randint(2, 4))
                soup = BeautifulSoup(page.content, "html.parser")
                prod_arr2 = soup.find_all('script', attrs={'type': re.compile("text/javascript")})
                prod_arr2 = str(prod_arr2[1])
                prod_arr2 = prod_arr2[prod_arr2.index('([[')+1 : prod_arr2.index(']);')-3]+']'
                prod_arr2 = literal_eval(prod_arr2)
                for i in range(len(prod_arr2)):
                    prod_arr2[i][2] = prod_arr2[i][2][prod_arr2[i][2].index('\n')+2:]
                    prod_arr2[i][4] = prod_arr2[i][4][prod_arr2[i][4].index('\n')+2:]
                    prod_arr2[i] = prod_arr2[i][0].split("-") + prod_arr2[i][1:]
                category_URL_arr3[Key_Layer1][Key_Layer2][Key_Layer3][Key_Layer4] = prod_arr2


***Put all data to CSV file:***

In [ ]:
column_names = ["Category",     "Tab",      "Sub Tub",      "Products",
                "Year",         "Month",    "High_Price",   "High_Place", "Low_Price", "Low_Place"]
df = pd.DataFrame(columns=column_names)
for Key_Layer1 in category_URL_arr3.keys():
    for Key_Layer2 in category_URL_arr3[Key_Layer1].keys():
        for Key_Layer3 in category_URL_arr3[Key_Layer1][Key_Layer2].keys():
            for Key_Layer4 in category_URL_arr3[Key_Layer1][Key_Layer2][Key_Layer3]:
                for data in category_URL_arr3[Key_Layer1][Key_Layer2][Key_Layer3][Key_Layer4]:
                    data_dict = {column_names[-1*len(data)+index]: data[index] for index in range(len(data))}
                    df = df.append(data_dict, ignore_index=True)
                df[column_names[-7]] = df[column_names[-7]].fillna(Key_Layer4)
            df[column_names[-8]] = df[column_names[-8]].fillna(Key_Layer3)
        df[column_names[-9]] = df[column_names[-9]].fillna(Key_Layer2)
    df[column_names[-10]] = df[column_names[-10]].fillna(Key_Layer1)
pd.DataFrame(df).to_csv('./product_file.csv', encoding="ISO-8859-8")
